In [1]:
import pandas as pd
import numpy as np
import itertools

In [2]:
df = pd.read_csv("custom.csv")
df.head()

,Member_number,Date,itemDescription,year,month,day,day_of_week
0,1808,2015-07-21,A C D,2015,7,21,1
1,2552,2015-05-01,B C E,2015,5,1,4
2,2300,2015-09-19,A B C E,2015,9,19,5
3,2552,2015-05-01,B E,2015,5,1,4


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Member_number    4 non-null      int64 
 1   Date             4 non-null      object
 2   itemDescription  4 non-null      object
 3   year             4 non-null      int64 
 4   month            4 non-null      int64 
 5   day              4 non-null      int64 
 6   day_of_week      4 non-null      int64 
dtypes: int64(5), object(2)
memory usage: 352.0+ bytes


In [4]:
df = df.drop(columns=['Member_number', 'Date', 'year', 'month', 'day', 'day_of_week'])
n = df.size

In [5]:
#Cleaning the column
df['itemDescription'] = df['itemDescription'].str.split()

In [6]:
df.head()

,itemDescription
0,"[A, C, D]"
1,"[B, C, E]"
2,"[A, B, C, E]"
3,"[B, E]"


In [7]:
def items_remaining(df_column):
    items = set()
    for l in df_column:
        for item in l:
            items.add(item)

    #print(len(items))
    total_item_count = len(items)
    return items

def contains_in(item_set):
    count = 0
    for i in df['itemDescription']:
        if(set(item_set).issubset(set(i))):
            count += 1
    #print(count)
    return count

df_a = pd.DataFrame()
flag = 1

def apriori(i, support=0.05):
    global df_a
    if(i == 1):
        item_sets = list(items_remaining(df['itemDescription']))
        item_sets = list(itertools.combinations(item_sets, 1))
        df_a['item_set'] = item_sets
        #print((type(df_a['item_set'][0])))
    else:
        item_sets = list(items_remaining(df_a['item_set']))
        item_sets = list(itertools.combinations(item_sets, i))
        df_a = df_a[0:0]
        df_a['item_set'] = item_sets

    frequency = list()
    for i in item_sets:
        frequency.append(contains_in(i))
    #print(frequency)
    df_a['frequency'] = frequency
    df_a['support'] = df_a['frequency'] / n
    #print(df_a)
    df_a.drop(df_a[df_a['support'] < support].index, inplace=True)
    if(df_a.empty):
        flag = 0
        return flag
    print(df_a)
    print("*******************************************")
    return 1

i = 0
while(flag):
    flag = apriori(i+1, 0.5)
    i += 1

  item_set  frequency  support
1     (B,)          3     0.75
2     (E,)          3     0.75
3     (C,)          3     0.75
4     (A,)          2     0.50
*******************************************
  item_set  frequency  support
1   (B, C)          2     0.50
2   (B, E)          3     0.75
3   (A, C)          2     0.50
5   (C, E)          2     0.50
*******************************************
    item_set  frequency  support
2  (B, C, E)          2      0.5
*******************************************
